In [1]:
from votekit.ballot import Ballot

In [2]:

ballot = Ballot(ranking=[{"A"}, {"B"}, {"C"}], weight=3 / 2)
print(ballot)

print("\n--------------------------------\n")

ballot = Ballot(ranking=[{"A"}, {"B"}, {"C"}], weight=32)
print(ballot)


RankBallot
1.) A, 
2.) B, 
3.) C, 
Weight: 1.5

--------------------------------

RankBallot
1.) A, 
2.) B, 
3.) C, 
Weight: 32.0
